# DataCleaning (Class Data)

In [2]:
#import required modules:
import numpy as np
import pandas as pd
from datetime import datetime, timezone
import matplotlib.pyplot as plt
import matplotlib.cm as cm


#for the "heavy" work:
from dateutil.parser import parse
import matplotlib as mpl
from matplotlib.pylab import rcParams
from sklearn import preprocessing

import seaborn as sns
#import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose


#ACF and PACF plots:
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf
#from statsmodels.tsa.arima_model import ARIMA

from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler


## Description of the Class Data
#### The class has 2 imputs: A Pandas DataFrame and a name:

--> To create an instance ot the Class Data, use following lines:
```json
dataframe = pd.read_csv("PATH TO DATA")
name="DESIRED NAME"
LakeAegeri=Data(dataframe,name)
```

#### The Class has various methods:
##### 1. The INSTANCE_CLASS_DATA.calc_rolling_statistics(column,window,plot="n",save_plot="n")
This method will calculate the rolling statistics and outputs a plot.

If you don't want the plot plotted in the notebook, use show_plot="n".

If you want to save the plot in the /data/output/.. folder:
- save_plot="y" -> It asks you to input a name without a space.
- save_plot="CUSTOM_NAME" -> It creates a plot with the name CUSTOM_NAME in the specified folder
- save_plot="n": It will not save the created figure
If you want to define the rolling window, use window=INTEGER


##### 2. The INSTANCE_CLASS_DATA.check_stationarity(column)
This method will check the stationarity using the Augmented Dickey-Fuller Test (ADF). As an input use the column you want the stationarity checked on.


##### 3. INSTANCE_CLASS_DATA.detect_outlier(column,knn_imputation="y",PERIOD=24,display_plots="y"):
This method will clean the data with help of the seasonal decomposition of the specified column (column=COLUMN_NAME). The method returns a (TimeSeries) cleaned DataFrame with the selected column. The physical impossible outlier are not cleaned in this method.

If you want the more "fancy" K-nearest Neighbours Algorithm Imputation, use knn_imputation="y". It looks at the 4 neighbours of the outlier and calculates an optimal value to replace the outlier. Otherwise, use knn_imputation="n" to use the simple imputer: It looks at the normalized residuals and sets >3 values to 3 and <-3 values to -3. In the last step.

PERIOD: If you want to change the period of the data, change the PERIOD parameter.

Display plots in the notebook: If you want to display the plots, use display_plots="y", else display_plots="n"

##### 4. INSTANCE_CLASS_DATA.outlier_physical()...

In [3]:
class Data:
    def __init__(self,dataframe,name):
        self.name=name
        self.dataframe= dataframe
        self.columns=self.dataframe.columns
        #convert the strings and add the datetime column to index
        self.dataframe["datetime"]=pd.to_datetime(self.dataframe["datetime"])
        self.dataframe.index=self.dataframe["x"]
        self.dataframe=self.dataframe.drop(columns=["x"])


    def __repr__(self):
        return f"DataFrame {name} with columns: {str(list(self.columns))}"

    def calc_rolling_statistics(self,column,window,show_plot="no",save_plot="no"):
        mean_rm = self.dataframe[column].rolling(window=window).mean()
        std_rm = self.dataframe[column].rolling(window=window).std()

        plt.style.context
        plt.rc('figure',figsize=(10,6))
        fig, ax = plt.subplots(1,sharex=False,sharey=False)
        ax.plot(self.dataframe["datetime"],self.dataframe[column], color='red', label='Actual',alpha=0.7)
        ax.plot(self.dataframe["datetime"],mean_rm, color='green', label=f'Rolling Mean (window: {window})',alpha=0.7)
        ax.plot(self.dataframe["datetime"],std_rm, color='black', label=f'Rolling Std (window: {window})',alpha=0.7)
        plt.legend(loc='best')
        plt.title(f'Rolling Mean & Standard Deviation of {column}')

        if save_plot =="y":
            plt.savefig(f"./data/output_png/{input('How should the figure be named? (no Spaces!)')}.png")
        elif save_plot != "no":
            plt.savefig(f"./data/output_png/{save_plot}.png")

        if show_plot=="n":
            plt.close()
        plt.show(block=False)

    def check_stationarity(self,column):
        print(f"Visual check is advised with {self.name}.calc_rolling_statsitics(...,plot='Y')")
        print("->  A stationary Time Series is one which characteristics like mean and variance does not change over time.")
        print("Another way of checking for stationarity of the time series is using Augmented Dickey-Fuller (ADF) Test to check stationarity:")
        print("--------------------")
        print('Dickey-Fuller Test: ')
        X = self.dataframe[column].dropna().values
        dftest=adfuller(X, autolag='AIC')
        dfoutput=pd.Series(dftest[0:4], index=['Test Statistic','p-value','Lags Used','No. of Obs'])
        for key,value in dftest[4].items():
            dfoutput['Critical Value (%s)'%key] = value

        if list(dfoutput.items())[1][1] > 0.05:
            print(f"with a p-value of {round(list(dfoutput.items())[1][1],3)}, we fail to reject the null hypothesis, that the data is stationary.")
            print(dfoutput)
            return False
        else:
            print(f"with a p-value of {round(list(dfoutput.items())[1][1],3)}, we reject the null hypothesis, that the data is stationary.")
            print(dfoutput)
            return True


    def detect_outlier(self,column,knn_imputation="y",PERIOD=24,display_plots="y"):
        ### 1. Prepare Dataset for the seasonal seasonal_decompose function:
        pd.set_option('display.max_rows', None)
        dataframe_noNAN=self.dataframe.dropna()
        dataframe_noNAN=dataframe_noNAN.reset_index()
        dataframe_noNAN=dataframe_noNAN.set_index("datetime")

        #set plot size:
        plt.rc('figure',figsize=(10,6))
        plt.rc('font',size=10)

        ### 2. Seasonal decompose
        result=seasonal_decompose(dataframe_noNAN[[column]],period=PERIOD)

        #plot the seseasonal_decompose:
        fig, ax = plt.subplots(4,sharex=False,sharey=False)
        fig.subplots_adjust(bottom=7,top=9)

        ax[0].plot(result.observed,color="blue")
        ax[0].set_title("Result observed")

        ax[1].plot(result.seasonal,color="red")
        ax[1].set_title(f"seasonality of Period {PERIOD}")

        ax[2].plot(result.trend,color="green")
        ax[2].set_title("Trend")

        ax[3].plot(result.resid,color="orange")
        ax[3].set_title("Residuals")

        if display_plots != "y":
            plt.close()
        plt.show()

        ## 2.1 residuals (imputation yes or no)
        seasonal_df=result.seasonal.to_frame()
        trend_df=result.observed.to_frame()
        residuals_df=result.resid.to_frame()

        # 2.1.1 Normalize residuals:
        mean = residuals_df.mean()
        std = residuals_df.std()
        residuals_df_norm=residuals_df.copy()
        residuals_df_norm = (residuals_df_norm[["resid"]] - mean[0])/std[0]

        # 2.1.2 If the normalized residuals exceed +/- 3, then this is considered an outlier:
        residuals_df_norm['is_outlier'] = np.where((residuals_df_norm['resid']> 3) | (residuals_df_norm['resid']<-3), True, False)
        print(f"------- number of outliers of {column} (normalized residuals exceed 3 or -3): ---------\n",residuals_df_norm["is_outlier"].value_counts())

        #plot exceeded residuals:
        fig, ax = plt.subplots()
        ax.scatter(residuals_df_norm.reset_index()["datetime"],residuals_df_norm.reset_index()["resid"],c=residuals_df_norm["is_outlier"],cmap=cm.Set2,s=1)
        ax.axhline(3,color="red")
        ax.axhline(-3,color="red")
        if display_plots != "y":
            plt.close()
        plt.show()

        # 2.1.3 IMPUTATION: if needed
        if knn_imputation=="y":
            print(f"---> With K-nearest Neighbours Algorithm Imputation of {column}. It looks at the neighbours of the outlier and calculates an optimal value to replace the outlier.\nThe function returns a (TimeSeries) cleaned DataFrame with the selected column {column}. The physical impossible outlier are not cleaned in this function.")
            #IMPUTATION:
            residuals_df_norm=residuals_df_norm.dropna()
            residuals_df_imputed=residuals_df_norm.copy()
            residuals_df_imputed['resid'] = np.where((residuals_df_imputed['resid']> 3) | (residuals_df_imputed['resid']<-3), np.nan, residuals_df["resid"].dropna())

            # Save datetime column and drop it (bc. imputer can't handle datetime)
            residuals_df_imputed=residuals_df_imputed.reset_index()
            datetime_col = residuals_df_imputed['datetime']
            residuals_df_imputed = residuals_df_imputed.drop('datetime', axis=1)

            # Drop any rows with missing values (bc. imputer can't handle NaN's)
            residuals_df_imputed=residuals_df_imputed.reset_index()
            residuals_df_imputed = residuals_df_imputed.dropna()

            # Impute missing values using KNNImputer (K-nearest Neighbours Algorithm)
            imputer = KNNImputer(n_neighbors=4)
            residuals_imputed = pd.DataFrame(imputer.fit_transform(residuals_df_imputed), columns=residuals_df_imputed.columns)

            # Add datetime column back and set it as index (see above)
            residuals_imputed['datetime'] = datetime_col
            residuals_imputed = residuals_imputed.set_index('datetime')

            residuals_cleaned=residuals_imputed.drop(columns=["is_outlier","index"])




        else:  #ASSUMING THAT ALL RESIDUALS WILL BE SET TO 3 or -3 (normalized) WHEN THEY ARE ABOVE 3 OR BELOW -3
            print(f"---> With simple imputation of {column}: It looks at the normalized residuals and sets >3 values to 3 and <-3 values to -3.\nThe function returns a (TimeSeries) cleaned DataFrame with the selected column {column}. The physical impossible outlier are not cleaned in this function. ")

            residuals_df_norm=residuals_df_norm.dropna()
            residuals_df_norm.loc[residuals_df_norm["resid"] > 3,"resid"] = 3
            residuals_df_norm.loc[residuals_df_norm["resid"] < -3,"resid"] = -3
            residuals_df_norm=residuals_df_norm.drop(columns=["is_outlier"])

        # De-Normalize (inverse of residuals_df_norm = (residuals_df_norm[["resid"]] - mean[0])/std[0])
            residuals_cleaned=residuals_df_norm.mul({"resid":std[0]}).add({"resid":mean[0]})


    #Compose the Timeseries again (by adding the components residuals, seasonality and trend)
        fig, ax = plt.subplots()
        ax.scatter(residuals_cleaned.reset_index()["datetime"],residuals_cleaned.reset_index()["resid"],s=1)
        if display_plots != "y":
            plt.close()
        plt.show()

        #merging the timestamps of the 3 DataFrame's
        data_cleaned= residuals_cleaned.join(seasonal_df).join(trend_df)
        data_cleaned[column]=data_cleaned["seasonal"]+data_cleaned["resid"]+data_cleaned[0]

        return data_cleaned[column]

    def remove_outliers_temperature(self):

        # Remove any rows where either "skinTemp" or "bulkTemp" is above 35
        df_cleaned = self.dataframe[(self.dataframe['y'] <= 35) & (self.dataframe['y1'] <= 35)]
        # Return the cleaned DataFrame
        return df_cleaned


        return dataframe_filtered.head(10000)

    def remove_outliersWind(self):
        # Calculate the z-score for the windspeed column
        z_scores = np.abs((self.dataframe['wind_speed'] - self.dataframe['wind_speed'].mean()) / self.dataframe['wind_speed'].std())
        # Create a new DataFrame with the outliers removed
        wind_speed_clean = self.dataframe[z_scores <= 3].copy()
        # Remove the corresponding time value for each removed row
        wind_speed_clean = wind_speed_clean.drop(wind_speed_clean[z_scores > 3].index)

        # Return the cleaned DataFrame
        return wind_speed_clean

In [4]:
dataframe = pd.read_csv("./data/df_aegeri_skin.csv")
name="LakeAegeri raw"

LakeAegeri=Data(dataframe,name)

In [5]:
LakeAegeri.detect_outlier("y1",PERIOD=144,display_plots="no",knn_imputation="y") #144 is 1 day in Aegeri(Outliers.loc['2022-12-02'])

------- number of outliers of y1 (normalized residuals exceed 3 or -3): ---------
 False    16581
True       340
Name: is_outlier, dtype: int64
---> With K-nearest Neighbours Algorithm Imputation of y1. It looks at the neighbours of the outlier and calculates an optimal value to replace the outlier.
The function returns a (TimeSeries) cleaned DataFrame with the selected column y1. The physical impossible outlier are not cleaned in this function.


datetime
2022-11-12 04:30:12.686    12.592227
2022-11-12 04:40:12.616    12.697737
2022-11-12 04:50:12.664    12.659428
2022-11-12 05:00:12.729    12.621873
2022-11-12 05:10:12.670    12.652389
2022-11-12 05:20:12.641    12.680517
2022-11-12 05:30:12.613    12.642085
2022-11-12 05:40:12.624    12.665973
2022-11-12 05:50:12.632    12.466600
2022-11-12 06:00:12.613    12.610552
2022-11-12 06:10:12.609    12.607550
2022-11-12 06:20:12.587    12.660734
2022-11-12 06:30:12.613    12.504030
2022-11-12 06:40:12.660    12.528649
2022-11-12 06:50:12.674    12.664512
2022-11-12 07:00:12.713    12.617907
2022-11-12 07:10:12.729    12.663343
2022-11-12 07:20:12.603    12.638894
2022-11-12 07:30:12.530    12.643846
2022-11-12 07:40:12.661    12.624902
2022-11-12 07:50:12.623    12.445257
2022-11-12 08:00:12.621    12.571517
2022-11-12 08:10:12.648    12.608430
2022-11-12 08:20:12.624    12.521837
2022-11-12 08:30:12.614    12.587743
2022-11-12 08:40:12.594    12.663059
2022-11-12 08:50:12.800    12

In [6]:
LakeAegeri.calc_rolling_statistics(window=100,column="y",show_plot="n",save_plot="rolling_mean")
LakeAegeri.calc_rolling_statistics(window=100,column="air_temp",show_plot="n")